# Cogroup

mete los grupos todos juntos<br/>
ej contar el número máximo de cupones por ticket.<br/>
<br/>
T1 C1  | <br/>
T1 C2  | --> (T1, [c1,c2,c3])<br/>
T2 C3  | --> (T2, [c3,c4])<br/>
T2 C4  |<br/>
T1 C5  |<br/>

In [2]:
couponPath = "/home/dsc/Data/spark/data/coupon150720.csv"
transmPath = "/home/dsc/Data/spark/data/transm150720.csv"

In [3]:
tcnCs = sc.textFile(couponPath).map(lambda x:x.split(",")[0])

In [4]:
cs = tcnCs.map(lambda x: (x,1))

In [18]:
cs.take(2)

[(u'79062005698500', 1), (u'79062005698500', 1)]

In [5]:
tcnTs = sc.textFile(transmPath).map(lambda x:x.split(",")[0])

In [6]:
ts = tcnTs.map(lambda x: (x,1))

In [7]:
ts.take(2)

[(u'79062005698500', 1), (u'79062005924069', 1)]

In [12]:
j = ts.cogroup(cs)

In [13]:
j.mapValues(lambda x: (len(x[0]), len(x[1]) )).take(3)

[(u'79062005994272', (1, 2)),
 (u'79065668614400', (1, 2)),
 (u'79062005879562', (1, 2))]

In [17]:
#Vamos a comprobar que las claves no estén duplicadas.
j.mapValues(lambda x: (len(x[0]), len(x[1]) )).map(lambda x: x[1][0]) \
.distinct().take(2)

[1]

In [19]:
j.mapValues(lambda x: (len(x[0]), len(x[1]) )) \
.sortBy(lambda x: x[1][1],False).take(5)

[(u'79065669300201', (1, 16)),
 (u'79062005712937', (1, 15)),
 (u'79062005779716', (1, 13)),
 (u'79065668940838', (1, 12)),
 (u'79062005319376', (1, 12))]

# Spark para Python no soporta clases de Python

In [ ]:
#Fecha de compra y fecha de vuelo
#histograma para ver la dif entre la compra y el vuelo.

# Ejercicio
Antelación de compra

In [21]:
cs = sc.textFile(couponPath)

In [23]:
#150904 -> fecha
cs.take(1)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0']

In [29]:
from time import strptime
from time import mktime

In [30]:
d1 = strptime("150904","%y%m%d")
d2 = strptime("150623","%y%m%d")

In [36]:
(mktime(d1)-mktime(d2))/(3600*24)

73.0

In [28]:
#La fecha de la compra es 141025
ts = sc.textFile(transmPath)
ts.first()

u'79062005698500,TKTT,30,150719,FR,0.0,EUR,T,T,141025,PARA127A8,0.0,EUR,   ,EX,,150719,0.0'

# Tabla
Trans -> fecha -> compra <br/>
cpns -> min(fecha) -> salida

In [38]:
ts.first()

u'79062005698500,TKTT,30,150719,FR,0.0,EUR,T,T,141025,PARA127A8,0.0,EUR,   ,EX,,150719,0.0'

In [67]:
def getTrans(l):
    elm = l.split(",")
    tcn = elm[0]
    bookingDate = mktime(strptime(elm[16],"%y%m%d"))
    return (tcn,bookingDate)

tsSelection = ts.map(getTrans)

In [56]:
cs.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [61]:
def getCoupons(l):
    elm = l.split(",")
    tcn = elm[0]
    fDate = mktime(strptime(elm[12],"%y%m%d"))
    return (tcn,fDate)

In [65]:
csSelection = cs.map(getCoupons)

In [68]:
tsSelection.take(1)

[(u'79062005698500', 1437256800.0)]

In [69]:
csSelection.take(1)

[(u'79062005698500', 1441317600.0)]

In [75]:
def getTCNSalida(l):
    elems = l.split(",")
    tcn = elems[0]
    salida = elems[12]
    return (tcn,salida)

In [78]:
cs = sc.textFile(couponPath).map(getTCNSalida)
cs.take(2)

[(u'79062005698500', u'150904'), (u'79062005698500', u'150905')]

In [79]:
def getTCNCompra(l):
    elems = l.split(",")
    tcn = elems[0]
    compra = elems [9]
    return (tcn, compra)

In [80]:
ts = sc.textFile(transmPath).map(getTCNCompra)
ts.take(2)

[(u'79062005698500', u'141025'), (u'79062005924069', u'150719')]

In [84]:
#Con un cogroup por un lado tendré el conjunto de todos los tickets y el conjunto de todos los cupones por el otro lado.
def fechaMenor(ds):
    
    segs = [mktime(strptime(d,"%y%m%d")) for d in ds]
    
    return min(segs)

In [82]:
ts.cogroup(cs).take(2)

[(u'79062005994272',
  (<pyspark.resultiterable.ResultIterable at 0x7fc511d0ad10>,
   <pyspark.resultiterable.ResultIterable at 0x7fc511d0a390>)),
 (u'79065668614400',
  (<pyspark.resultiterable.ResultIterable at 0x7fc511d20510>,
   <pyspark.resultiterable.ResultIterable at 0x7fc511d20450>))]

In [91]:
ts.cogroup(cs).mapValues(lambda x: fechaMenor(x[1]) \
                         - fechaMenor(x[0]) \
                         ).take(1)

[(u'79062005994272', 86400.0)]

In [92]:
r = ts.cogroup(cs).mapValues(lambda x: fechaMenor(x[1]) \
                         - fechaMenor(x[0]))
    

In [93]:
r.take(2)

[(u'79062005994272', 86400.0), (u'79065668614400', 432000.0)]

In [95]:
fts = ts.filter(lambda x: x[1]!="")
fcs = cs.filter(lambda x: x[1]!="")


In [96]:
r = fts.cogroup(fcs).mapValues(lambda x: fechaMenor(x[1]) \
                         - fechaMenor(x[0]))
    

In [97]:
r.take(2)

[(u'79062005994272', 86400.0), (u'79065668614400', 432000.0)]

In [104]:
#Quiere hacer un histograma.

h=r.mapValues(lambda dif: dif/(3600*24)).map(lambda x: (int(x[1]),1)).reduceByKey(lambda x,y: x+y)

In [105]:
h.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 81.0 failed 1 times, most recent failure: Lost task 0.0 in stage 81.0 (TID 147, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2355, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2355, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py",